# Defining the Model Parameters

In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

# Define the model class
class Model_cointoss(nn.Module):
    def __init__(self):
        super(Model_cointoss, self).__init__()
        # Define the CNN layers
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU()
        )
        # Flatten layer
        self.flatten = nn.Flatten()
        # Dense layers
        self.dense_layers = nn.Sequential(
            nn.Linear(2560001, 128),  # +1 for the integer label, adjust the input size
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
        )
        # Output layers
        self.classification_output = nn.Linear(64, 1)  # Single neuron for binary classification
        self.regression_output = nn.Linear(64, 1)  # Single neuron for regression output

    def forward(self, x, label):
        x = self.conv_layers(x)
        x = self.flatten(x)
        combined = torch.cat((x, label.unsqueeze(1)), dim=1)
        combined = self.dense_layers(combined)
        class_output = torch.sigmoid(self.classification_output(combined))
        reg_output = self.regression_output(combined)
        return class_output, reg_output

# Function to preprocess the image
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        # transforms.Resize((128, 128)),  # Uncomment if used during training
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    return transform(image).unsqueeze(0)  # Add batch dimension

def predict(model, image_path, device):
    image = preprocess_image(image_path)
    image = image.to(device)
    initial_position = torch.tensor([0], device=device)
    with torch.no_grad():
        predicted_binary_label, predicted_distance_label = model(image, initial_position)
        # Apply threshold to convert to hard classification
        binary_label_class = 1 if predicted_binary_label.item() >= 0.5 else 0
        return binary_label_class, predicted_distance_label.item()

# Main execution
# if __name__ == "__main__":
#     model = Model31()
#     model.load_state_dict(torch.load('model31.pth'))
#     model.eval()
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)

#     binary_label, distance_label = predict(model, '/home/eren/Desktop/para/test/6_vert_8cm_tura.png', device)
#     print(f'Binary Label: {binary_label}, Distance Label: {distance_label}')

# Testing Model on New Images

In [3]:
import os

def predict_on_folder(model, folder_path, device):
    results = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):  # Check for image file extensions
            image_path = os.path.join(folder_path, filename)
            binary_label, distance_label = predict(model, image_path, device)
            results.append((filename, binary_label, distance_label))
    return results

# Main execution
if __name__ == "__main__":
    model = Model_cointoss()
    model.load_state_dict(torch.load('model31.pth'))
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    folder_path = '/home/eren/Desktop/para/test'
    test_results = predict_on_folder(model, folder_path, device)

    # Print all results
    for filename, binary_label, distance_label in test_results:
        print(f'{filename}, Binary Prediction (0 for heads, 1 for tails) : {binary_label}, Estimated Distance: {distance_label}')
        
def extract_info_from_filename(filename):
    parts = filename.split('_')
    # Extract true class
    true_class = 0 if parts[-1].split('.')[0] == 'tura' else 1
    # Extract true distance
    true_distance = float(parts[2].replace('cm', ''))
    return true_class, true_distance

def predict_on_folder(model, folder_path, device):
    results = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png')):
            true_class, true_distance = extract_info_from_filename(filename)
            image_path = os.path.join(folder_path, filename)
            predicted_binary_label, predicted_distance_label = predict(model, image_path, device)
            results.append((filename, true_class, true_distance, predicted_binary_label, predicted_distance_label))
    return results

9_tura_32cm_tura.png, Binary Prediction (0 for heads, 1 for tails) : 1, Estimated Distance: 14.629281997680664
11_tura_26cm_yazi.png, Binary Prediction (0 for heads, 1 for tails) : 0, Estimated Distance: 22.031442642211914
1_yazi_10cm_tura.png, Binary Prediction (0 for heads, 1 for tails) : 1, Estimated Distance: 15.02271556854248
5_vert_18cm_yazi.png, Binary Prediction (0 for heads, 1 for tails) : 1, Estimated Distance: 21.24978256225586
6_vert_8cm_tura.png, Binary Prediction (0 for heads, 1 for tails) : 1, Estimated Distance: 43.62507247924805
3_yazi_81cm_tura.png, Binary Prediction (0 for heads, 1 for tails) : 1, Estimated Distance: 23.65674591064453
12_tura_74cm_yazi.png, Binary Prediction (0 for heads, 1 for tails) : 1, Estimated Distance: 44.77896499633789
7_vert_41cm_yazi.png, Binary Prediction (0 for heads, 1 for tails) : 1, Estimated Distance: 21.930335998535156
10_tura_11cm_yazi.png, Binary Prediction (0 for heads, 1 for tails) : 1, Estimated Distance: 24.372745513916016
8_ve

# Evaluation of the Model

In [13]:
import numpy as np
from sklearn.metrics import confusion_matrix, mean_squared_error

def calculate_metrics(test_results):
    true_classes = []
    predicted_classes = []
    true_distances = []
    predicted_distances = []

    for _, true_class, true_distance, predicted_binary_label, predicted_distance_label in test_results:
        true_classes.append(true_class)
        predicted_classes.append(predicted_binary_label)
        true_distances.append(true_distance)
        predicted_distances.append(predicted_distance_label)

    return conf_matrix, mse

# Main execution
if __name__ == "__main__":
    test_results = predict_on_folder(model, '/home/eren/Desktop/para/test', device)

# Extracting the relevant columns
actual_classes = [row[1] for row in test_results]
predicted_classes = [row[3] for row in test_results]
actual_x_values = [row[2] for row in test_results]
predicted_x_values = [row[4] for row in test_results]

# Creating the confusion matrix
conf_matrix = confusion_matrix(actual_classes, predicted_classes)

# Calculating the MSE
mse = mean_squared_error(actual_x_values, predicted_x_values)

print("Confusion Matrix:")
print(conf_matrix)
print("Mean Squared Error for regression:", mse)

Confusion Matrix:
[[0 6]
 [1 5]]
Mean Squared Error for regression: 633.1439117202684
